### Convert orbital paramters to TLE

In [1]:
import datetime
from datetime import timezone
import skyfield
from skyfield.api import EarthSatellite, load, wgs84, N, S, E, W
import numpy as np


In [2]:
## Charge le module 'temps'
ts = load.timescale()

In [4]:
# 2023-11-09T19:43:22.890Z  

epo_year = 2024
epo_month = 7
epo_day = 9
epo_hour = 19
epo_min = 00
epo_sec = 00
epo_mil = 00


launch_epoch = datetime.datetime(epo_year, epo_month, epo_day, epo_hour,epo_min, epo_sec, epo_mil)
begining_of_the_world = datetime.datetime(1949, 12, 31, 00, 00,00,00)

In [5]:
epoch = launch_epoch - begining_of_the_world
epoch = epoch.total_seconds()/(60*60*24)
epoch

27219.791666666668

$$ eccentricité = \frac{r_a - r_p}{r_a + r_p} $$
$$ P = 2*\pi * \sqrt{\frac{a^3}{GMt}}$$
$$ n = \frac{2*\pi}{P}$$

In [6]:
def periodOrbit(nu, a):
    return round(2*np.pi*(((a**3)/nu))**(1/2),5)

In [10]:
## A RENSEIGNER
ra = 579.620 # [km]
rp = 575.072 # [km]

In [12]:

r_terre = 6378.137 # [km]
a = ((rp + ra)/2+ r_terre )* 1000 # [m]

G=6.67e-11 #cst grav. [m3/kg/s2]
Mt=5.97e24 #masse terre [kg]
nu=G*Mt #[m3/s2]

P = periodOrbit(nu, a) / 60
sat_ecco = (ra - rp)/(ra + rp)
no_kozai = (2*np.pi / P)

print("alt. orbite moyenne (a) [km] : ", round(a/1000 - 6378.14,3), "\necc : ", sat_ecco, "\nperiod [min] :",P, "\nmean motion [rad/min] : ",no_kozai )

alt. orbite moyenne (a) [km] :  577.343 
ecc :  0.003938712661038616 
period [min] : 96.265235 
mean motion [rad/min] :  0.06526951611534097


In [13]:
# Drag coef
C_D = 1.05 # 1.05 for a cube, 0.8 for a angled cube
A_s = 0.1 *0.1 # the cross-sectional area
m = 1 # mass of satellite, kg
rho_air = 1.225 # kg/m3 

B = C_D * A_s / m
B_star = B * rho_air / 2

print("B* : ", B_star)

B* :  0.006431250000000002


In [131]:
sat_num = 99899            # ok 	                        	# satnum: Satellite number (5 digit)
sat_epoch = epoch      # ok                            	# epoch: days since 1949 December 31 00:00 UT
sat_bstar = B_star     # ok                             	# bstar: drag coefficient (/earth radii)
sat_ndot = 0	                        	           # ndot: ballistic coefficient (revs/day)
sat_nndot = 0.0                                        # nddot: second derivative of mean motion (revs/day^3)
# sat_ecco = 0.1859667    	                           # ecco: eccentricity
sat_argpo = ( 360 -131.945 )  * (np.pi / 180)                   # argpo: argument of perigee (radians)
inclo = 97.459 * (np.pi / 180)                        # inclo: inclination (radians)
mo = (360 -157.843 )    * (np.pi / 180)                                      	# mo: mean anomaly (radians)
# no_kozai = 0.0472294454407 	                           # no_kozai: mean motion (radians/minute)
nodeo = 34.405	* (np.pi / 180)                    	# nodeo: right ascension of ascending node (radians)

In [132]:
from sgp4.api import Satrec, WGS72, WGS84

satrec = Satrec()
satrec.sgp4init(
    WGS84,              # gravity model
    'i',                # 'a' = old AFSPC mode, 'i' = improved mode
    sat_num,          	 	# satnum: Satellite number
    sat_epoch,       	    # epoch: days since 1949 December 31 00:00 UT
    sat_bstar,      	    # bstar: drag coefficient (/earth radii)
    sat_ndot,	 	        # ndot: ballistic coefficient (revs/day)
    sat_nndot,             	# nddot: second derivative of mean motion (revs/day^3)
    sat_ecco,    	        # ecco: eccentricity
    sat_argpo,	            # argpo: argument of perigee (radians)
    inclo,		            # inclo: inclination (radians)
    mo, 	                # mo: mean anomaly (radians)
    no_kozai, 	            # no_kozai: mean motion (radians/minute)
    nodeo, 		            # nodeo: right ascension of ascending node (radians)
)

In [133]:
sat = EarthSatellite.from_satrec(satrec, ts)
print('Satellite number:', sat.model.satnum)
print('Epoch:', sat.epoch.utc_jpl())

Satellite number: 88888
Epoch: A.D. 2023-Dec-01 19:42:16.0010 UTC


In [134]:
sat

<EarthSatellite catalog #88888 epoch 2023-12-01 19:42:16 UTC>

In [135]:
sat.model.twoline2rv

<function Satrec.twoline2rv>

### Print TLE

In [136]:
sat_name = 'ENSO'

In [137]:
from sgp4.api import jday
jd, fr = jday(epo_year,epo_month,epo_day,epo_hour,epo_min,epo_sec)
print(jd, fr)

2460279.5 0.8210185185185185


In [138]:
sat.model.satnum 

88888

In [139]:
from sgp4 import exporter
line1, line2 = exporter.export_tle(satrec)
print(line1,"\n", line2)

1 88888U          23335.82101853  .00000000  00000-0  64313-2 0    06 
 2 88888  97.4590  34.4050 0061006 228.0550 202.1570 15.16981528    00


In [108]:
sat

<EarthSatellite catalog #88888 epoch 2023-12-01 18:11:07 UTC>

In [109]:
t = ts.utc(epo_year, epo_month, epo_day, epo_hour, epo_min, epo_sec)

In [110]:
from skyfield.framelib import itrs, ICRS

In [111]:
from skyfield.framelib import \
    true_equator_and_equinox_of_date as of_date

In [112]:
sat.at(t).frame_xyz(itrs).km

array([2160.17669862, 1348.27411255, 6438.30638562])

In [113]:
## see API doc
# sat.at(t).frame_latlon_and_rates(itrs)
sat.at(t).frame_latlon(itrs)

(<Angle +68deg 25' 14.7">, <Angle 31deg 58' 13.2">, <Distance 4.62813e-05 au>)

In [114]:
## PERMET DE PASSER D'UN REFERENTIEL ICRS → ITRS @ epoch 't'
R = itrs.rotation_at(t)
R

array([[ 9.55365231e-01, -2.95419104e-01, -2.19737567e-03],
       [ 2.95418245e-01,  9.55367757e-01, -7.13343330e-04],
       [ 2.31003712e-03,  3.23585508e-05,  9.99997331e-01]])

In [115]:
sat.at(t)

<Geocentric ICRS position and velocity at date t center=399 target=-188888>

In [116]:
from skyfield.elementslib import OsculatingElements

In [117]:
r_ecef_m =  [4445128.808, 4530167.679, -2720070.099]
v_ecef_m_per_s = [3174.025, 1038.424, 6914.022]